In [17]:
from typing import Self, Callable
import re
from tqdm.notebook import tqdm
import numpy as np

In [18]:
def load_data(filename):
    with open(filename, 'r') as f:
        data = f.read().split('\n\n')
    return data

In [19]:
class Monkey():
    
    def __init__(self,
                 number: int,               # number of this monkey
                 items: list[int],          # list of items this monkey is carrying
                 operation: Callable[int, int],  # operation i [int] -> j [int]
                 test: int, 
                 if_true: int,              # number of the monkey to throw to if True
                 if_false: int) -> None:    # number of the monkey to throw to if False
        self.number = number
        self.items = items
        self.operation = operation
        self.test = test
        self.if_true = if_true
        self.if_false = if_false
        self.inspected = 0
        
    def divisible_by_test(self, n):
        return n % self.test == 0
        
    @classmethod
    def from_input_data(cls, d) -> Self:
        lines = [line.strip() for line in d.split('\n')]
        number = int(lines[0].split(' ')[1].split(':')[0])
        items = [int(s.split(',')[0]) for s in lines[1].split(' ')[2:]]
        operation = lines[2].split(' ')[4:]
        if operation[0] == '+':
            operation_func = lambda x: x + int(operation[1])
        elif operation[0] == '*':
            if operation[1] == 'old':
                operation_func =  lambda x: x * x
            else:
                operation_func = lambda x: x * int(operation[1])
        else:
            raise ValueError
        divisible_by = int(lines[3].split(' ')[-1])
        test = divisible_by
        if_true = int(lines[4].split(' ')[-1])
        if_false = int(lines[5].split(' ')[-1])
        return Monkey(number=number,
                      items=items,
                      operation=operation_func,
                      test=test,
                      if_true=if_true,
                      if_false=if_false)

In [20]:
filename = "AoC day 11 example data.txt"
# filename = "AoC day 11 data.txt"
data = load_data(filename)

In [21]:
monkeys = {}
for d in data:
    m = Monkey.from_input_data(d)
    monkeys[m.number] = m

In [22]:
def mim_round(monkeys):
    for m in monkeys.values():
#     print(f"- Current monkey: {m.number}\n")
        thrown_to = {i: [] for i in monkeys.keys()}
        for item in m.items:
            m.inspected += 1
    #         print(f'Worry level: {item}')
            new_worry_level = m.operation(item)
    #         print(f'New worry level: {new_worry_level}')
            test_value = new_worry_level // 3
    #         print(f'Monkey is bored: {test_value}')
            test_outcome = m.divisible_by_test(test_value)
    #         print(f'{test_outcome}')
            thrown_to = m.if_true if test_outcome else m.if_false
    #         print(f'Thrown to monkey {thrown_to}')
            monkeys[thrown_to].items.append(test_value)
    #         print()
        m.items = []
# end of round:
    for m in monkeys.values():
        print(f"Monkey {m.number}: {', '.join([str(i) for i in m.items])}")

In [23]:
for i in range(20):
    mim_round(monkeys)

Monkey 0: 20, 23, 27, 26
Monkey 1: 2080, 25, 167, 207, 401, 1046
Monkey 2: 
Monkey 3: 
Monkey 0: 695, 10, 71, 135, 350
Monkey 1: 43, 49, 58, 55, 362
Monkey 2: 
Monkey 3: 
Monkey 0: 16, 18, 21, 20, 122
Monkey 1: 1468, 22, 150, 286, 739
Monkey 2: 
Monkey 3: 
Monkey 0: 491, 9, 52, 97, 248, 34
Monkey 1: 39, 45, 43, 258
Monkey 2: 
Monkey 3: 
Monkey 0: 15, 17, 16, 88, 1037
Monkey 1: 20, 110, 205, 524, 72
Monkey 2: 
Monkey 3: 
Monkey 0: 8, 70, 176, 26, 34
Monkey 1: 481, 32, 36, 186, 2190
Monkey 2: 
Monkey 3: 
Monkey 0: 162, 12, 14, 64, 732, 17
Monkey 1: 148, 372, 55, 72
Monkey 2: 
Monkey 3: 
Monkey 0: 51, 126, 20, 26, 136
Monkey 1: 343, 26, 30, 1546, 36
Monkey 2: 
Monkey 3: 
Monkey 0: 116, 10, 12, 517, 14
Monkey 1: 108, 267, 43, 55, 288
Monkey 2: 
Monkey 3: 
Monkey 0: 91, 16, 20, 98
Monkey 1: 481, 245, 22, 26, 1092, 30
Monkey 2: 
Monkey 3: 
Monkey 0: 162, 83, 9, 10, 366, 12, 34
Monkey 1: 193, 43, 207
Monkey 2: 
Monkey 3: 
Monkey 0: 66, 16, 71
Monkey 1: 343, 176, 20, 22, 773, 26, 72
Monkey 2: 

In [24]:
for m in monkeys.values():
    print(f'Monkey {m.number} inspected items {m.inspected} times.')

Monkey 0 inspected items 101 times.
Monkey 1 inspected items 95 times.
Monkey 2 inspected items 7 times.
Monkey 3 inspected items 105 times.


In [25]:
n_inspected = sorted([m.inspected for m in monkeys.values()])

In [26]:
monkey_business = n_inspected[-2] * n_inspected[-1]
print(monkey_business)

10605


In [27]:
# part 2

In [45]:
def mim_round(monkeys):
    supermod = np.prod([m.test for m in monkeys.values()])
    for m in monkeys.values():
        thrown_to = {i: [] for i in monkeys.keys()}
        for item in m.items:
            m.inspected += 1
            new_worry_level = m.operation(item) % supermod
            test_value = new_worry_level
            test_outcome = m.divisible_by_test(test_value)
            thrown_to = m.if_true if test_outcome else m.if_false
            monkeys[thrown_to].items.append(test_value)
        m.items = []
# # end of round:
#     for m in monkeys.values():
#         print(f"Monkey {m.number}: {', '.join([str(i) for i in m.items])}")

In [52]:
# filename = "AoC day 11 example data.txt"
filename = "AoC day 11 data.txt"
data = load_data(filename)

In [53]:
monkeys = {}
for d in data:
    m = Monkey.from_input_data(d)
    monkeys[m.number] = m

In [54]:
for i in tqdm(range(10000)):
    mim_round(monkeys)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [55]:
for m in monkeys.values():
    print(f'Monkey {m.number} inspected items {m.inspected} times.')

Monkey 0 inspected items 45924 times.
Monkey 1 inspected items 131087 times.
Monkey 2 inspected items 87798 times.
Monkey 3 inspected items 95174 times.
Monkey 4 inspected items 107420 times.
Monkey 5 inspected items 96174 times.
Monkey 6 inspected items 74384 times.
Monkey 7 inspected items 41580 times.


In [56]:
n_inspected = sorted([m.inspected for m in monkeys.values()])

In [57]:
monkey_business = n_inspected[-2] * n_inspected[-1]
print(monkey_business)

14081365540
